This is where I learn how to use cv2(opencv) to work with video

Add library for work

In [ ]:
# pip install opencv-python
# pip install ultralytics
# pip install supervision

In [3]:
pip install supervision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.5/181.5 kB 3.1 MB/s eta 0:00:00


In [4]:
import cv2

Connect to google drive

In [5]:
from google.colab import drive
drive.mount('/content/drive')

!ls "/content/drive/My Drive"

# !fusermount -u /content/drive
# !rm -rf /content/drive

Mounted at /content/drive
'10-Nguyễn Công Thành.mkv'			   HTML
'2402_Buoi03_ThanhNC(Tren_Lop).sql'		   Java
 Android2.drawio				   K16_Testing11_nhom10
 Android.drawio					  'K16_Testing11_nhom10 (1)'
 ArtStoreProject.rar				  'Kiem thu'
 Bài-tập-thuyết-trình-về-khu-vực.pptx		  'LeagueClient Logs.rar'
 BBKH_TruongCongManh.docx			   Maven
 BCKH_TruongCongManh.docx			   NgoaiNgu
'Biểu đồ ca sử dụng.gdoc'			  'Nguyễn Công Thành.pdf'
'BTL Học máy'					  'Nhóm 14 TTCSN'
 BTL_N1_HocMay_final.docx			   Presentation4.pptx
'Buổi 1 - Giới thiệu HP gửi SV.pptx'		  'Professional statement exemplar.gdoc'
'Chương 1_2 MỘT SỐ GỢI Ý ĐỂ HỌC TỐT (1).gslides'  'Professional statement outline.gdoc'
'Chương 1_2 MỘT SỐ GỢI Ý ĐỂ HỌC TỐT (2).gslides'   Python
'Chương 1_2 MỘT SỐ GỢI Ý ĐỂ HỌC TỐT.gslides'	   Python.drawio
'Chương 2 Cơ bản về MT - CNTT (1).gslides'	   SQL
'Chương 2 Cơ bản về MT - CNTT (2).gslides'	  'Tên ca

Read and save video

In [26]:
def read_video(path):
  cap = cv2.VideoCapture(path)
  if not cap.isOpened():
    print("Error opening video stream or file")
    return
  frames = []
  while True:
    ret, frame = cap.read()
    if not ret:
      break
    frames.append(frame)
  cap.release()
  cv2.destroyAllWindows()
  return frames

def save_video(frames, path):
  fourcc = cv2.VideoWriter_fourcc(*'mp4v')
  out = cv2.VideoWriter(path, fourcc, 20.0, (frames[0].shape[1], frames[0].shape[0]))
  for frame in frames:
    out.write(frame)
  out.release()

Tracking

In [27]:
def get_center_of_box(bbox):
  x1, y1, x2, y2 = bbox
  return int((x1+x2)/2), int((y1+y2)/2)

def get_bbox_wdth(bbox):
  x1, y1, x2, y2 = bbox
  return x2-x1

Main Class

In [28]:
from ultralytics import YOLO
import supervision as sv
import numpy as np
import pickle
import os
import cv2

class Tracker:
  def __init__(self, model_path):
    self.model = YOLO(model_path)
    self.tracker = sv.ByteTrack()

  def detect_frames(self, frames):
    batch_size = 20
    detection = []
    for i in range(0, len(frames), batch_size):
      detection_batch = self.model.predict(frames[i:i+batch_size], conf= 0.1)
      detection += detection_batch
      # batch = frames[i:i+batch_size]
      # detection.append(self.model(batch))

    return detection

  def draw_annotations(self, video_frames, tracks):
      output_video_frames = []
      for frame_num, frame in enumerate(video_frames):
        frame = frame.copy()

        player_dict = tracks["players"][frame_num]
        referee_dict = tracks["referees"][frame_num]
        ball_dict = tracks["ball"][frame_num]


        #Draw players
        for track_id, player in player_dict.items():
          frame = self.draw_eclipse(frame, player["bbox"], (0,0,255), track_id)

        #Draw referees
        for _, referee in referee_dict.items():
          frame = self.draw_eclipse(frame, referee["bbox"], (0,255,255))

        #Draw ball
        for track_id, ball in ball_dict.items():
          frame = self.draw_triangle(frame, ball["bbox"], (0,255,0))

        output_video_frames.append(frame)

      return output_video_frames

  def draw_eclipse(self, frame, bbox, color, track_id=None):
      y2 = int(bbox[3])

      x_center, _ = get_center_of_box(bbox)
      width = get_bbox_wdth(bbox)

      cv2.ellipse(
          frame,
          center=(x_center, y2),
          axes=(int(width), int(0.35*width)),
          angle=0,
          startAngle=-45,
          endAngle=235,
          color=color,
          thickness=2,
          lineType=cv2.LINE_4
      )

      rectangle_width = 40
      rectangle_height = 20
      x1_rect = x_center - rectangle_width // 2
      y1_rect = (y2 - rectangle_height//2) + 15
      x2_rect = x_center + rectangle_width // 2
      y2_rect = (y2 + rectangle_height//2) + 15

      if track_id is not None:
        cv2.rectangle(
            frame,
            (int(x1_rect), int(y1_rect)),
            (int(x2_rect), int(y2_rect)),
            color,
            cv2.FILLED)
        x1_text = x1_rect + 12
        if track_id > 99:
          x1_text -= 10

        cv2.putText(
            frame,
            str(track_id),
            (int(x1_text), int(y1_rect+25)),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.6,
            (0,0,0),
            2
            )

      return frame


  def draw_triangle(self, frame, bbox, color):
    y = int(bbox[1])
    x,_ = get_center_of_box(bbox)

    triangle_points = np.array([
        [x,y],
        [x-10, y-20],
        [x+10, y-20]
    ])
    cv2.drawContours(frame, [triangle_points], 0, color, cv2.FILLED)
    cv2.drawContours(frame, [triangle_points], 0, (0,0,0), 2)

    return frame

  def get_object_tracks(self, frames, read_from_stub=False, stub_path=None): #Save the result

    if read_from_stub and stub_path is not None and os.path.exists(stub_path):
      with open(stub_path, 'rb') as stub_file:
        tracks = pickle.load(stub_file)
      return tracks

    detections = self.detect_frames(frames)


    tracks = {
        "players": [],
        "referees": [],
        "ball": []
    }

    for frame_num, detection in enumerate(detections):
      cls_names = detection.names
      cls_names_inv = {v: k for k, v in cls_names.items()}

      #Convert to supervision Detection format
      detection_supervision = sv.Detections.from_ultralytics(detection)

      #Convert Goalkeeper to player object

      for object_ind, class_id in enumerate(detection_supervision.class_id):
        if class_id == cls_names_inv['goalkeeper']:
          detection_supervision.class_id[object_ind] = cls_names_inv['player']

      # Track Objects
      detection_with_tracks = self.tracker.update_with_detections(detection_supervision)

      tracks["players"].append({})
      tracks["referees"].append({})
      tracks["ball"].append({})

      for frame_detection in detection_with_tracks:
        bbox = frame_detection[0].tolist()
        cls_id = frame_detection[3]
        track_id = frame_detection[4]

        if cls_id == cls_names_inv['player']:
          tracks["players"][frame_num][track_id] = {"bbox" : bbox}
        elif cls_id == cls_names_inv['referee']:
          tracks["referees"][frame_num][track_id] = {"bbox" : bbox}
        elif cls_id == cls_names_inv['ball']:
          tracks["ball"][frame_num][1] = {"bbox" : bbox}

    if stub_path is not None:
        with open(stub_path, 'wb') as stub_file:
          pickle.dump(tracks, stub_file)


    return tracks




Main

In [29]:
video_frames = read_video('drive/MyDrive/Colab Notebooks/YOLO/input_video/challenge-1003_2.mp4')
# save_video (video_frames, 'drive/MyDrive/Colab Notebooks/YOLO/challenge-1003_2.mp4')

In [31]:
tracker = Tracker('drive/MyDrive/Colab Notebooks/YOLO/train/Train4/best.pt')

tracks = tracker.get_object_tracks(video_frames,
                                   read_from_stub=True,
                                   stub_path='drive/MyDrive/Colab Notebooks/YOLO/tracking/track_stubs_1.pkl')

#Draw output
## Draw object Tracks
output_video_frames = tracker.draw_annotations(video_frames, tracks)

# save cropped image of a player
for track_id, player in tracks['players'][0].items():
  bbox = player['bbox']
  frame = video_frames[0]


  #crop bbox from frame
  cropped_image = frame[int(bbox[1]):int(bbox[3]), int(bbox[0]):int(bbox[2])]

  # save the cropped image
  cv2.imwrite(f'drive/MyDrive/Colab Notebooks/YOLO/output_video/images/player_{track_id}.jpg', cropped_image)
  break

save_video (output_video_frames, 'drive/MyDrive/Colab Notebooks/YOLO/output_video/challenge-1003_2.mp4')


0: 384x640 2 goalkeepers, 26 players, 2 referees, 512.8ms
1: 384x640 2 goalkeepers, 24 players, 2 referees, 512.8ms
2: 384x640 2 goalkeepers, 23 players, 2 referees, 512.8ms
3: 384x640 1 goalkeeper, 22 players, 2 referees, 512.8ms
4: 384x640 2 goalkeepers, 23 players, 1 referee, 512.8ms
5: 384x640 2 goalkeepers, 26 players, 2 referees, 512.8ms
6: 384x640 2 goalkeepers, 28 players, 2 referees, 512.8ms
7: 384x640 2 goalkeepers, 27 players, 2 referees, 512.8ms
8: 384x640 2 goalkeepers, 25 players, 2 referees, 512.8ms
9: 384x640 2 goalkeepers, 23 players, 2 referees, 512.8ms
10: 384x640 2 goalkeepers, 25 players, 1 referee, 512.8ms
11: 384x640 1 goalkeeper, 24 players, 1 referee, 512.8ms
12: 384x640 2 goalkeepers, 24 players, 1 referee, 512.8ms
13: 384x640 2 goalkeepers, 25 players, 1 referee, 512.8ms
14: 384x640 2 goalkeepers, 23 players, 1 referee, 512.8ms
15: 384x640 2 goalkeepers, 23 players, 1 referee, 512.8ms
16: 384x640 1 goalkeeper, 26 players, 1 referee, 512.8ms
17: 384x640 2 goa